# Imports

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os
from tqdm import tqdm
import random
from torchsummary import summary
import pandas as pd
import zipfile

# Testing of Siamese

In [13]:
class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        # Convolutional layers
        self.conv1 = nn.Conv2d(1, 8, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(8)
        self.conv2 = nn.Conv2d(8, 16, kernel_size=5, padding=0)
        self.bn2 = nn.BatchNorm2d(16)
        self.conv3 = nn.Conv2d(16, 32, kernel_size=3, padding=0)
        self.bn3 = nn.BatchNorm2d(32)

        # Fully connected layers
        self.fc1 = nn.Linear(32 * 12 * 12, 41)  # Updated to 32 * 12 * 12
        self.fc1komma5 = nn.Linear(41,32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 1)
        self.dropout = nn.Dropout(0.5)

    def forward_one(self, x):
        x = F.relu(self.conv1(x)) # 8 * 112 * 112
        x = F.max_pool2d(x, 2)  # output size: (8, 56, 56)
        x = F.relu(self.conv2(x)) # 16* 52 * 52
        x = F.max_pool2d(x, 2)  # output size: (16, 26, 26)
        x = F.relu(self.conv3(x)) # 32 * 24 * 24
        x = F.max_pool2d(x, 2)  # output size: (32, 12, 12)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc1komma5(x))
        x = F.relu(self.fc2(x))
        return x

    def forward(self, input1, input2):
        output1 = self.forward_one(input1)
        output2 = self.forward_one(input2)
        distance = torch.abs(output1 - output2)
        output = self.fc3(distance)
        return output

In [19]:
class RealTestDataset(Dataset):
    def __init__(self, image_folder, pairs_file, transform=None):
        self.image_folder = image_folder
        self.pairs = self._read_pairs(pairs_file)
        self.transform = transform

    def _read_pairs(self, pairs_file):
        pairs = []
        with open(pairs_file, 'r') as file:
            for line in file:
                name_1, number_1, name_2, number_2 = line.strip().split()
                img1_path = os.path.join(self.image_folder, name_1, f'{name_1}_{int(number_1):04d}.png')
                img2_path = os.path.join(self.image_folder, name_2, f'{name_2}_{int(number_2):04d}.png')
                label = 1 if name_1 == name_2 else 0
                pairs.append((img1_path.replace('\\', '/'), img2_path.replace('\\', '/'), label))
        return pairs

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        img1_path, img2_path, label = self.pairs[idx]
        try:
            img1 = Image.open(img1_path).convert('L')
            img2 = Image.open(img2_path).convert('L')
        except FileNotFoundError:
            # Handle missing image files
            print(f"Warning: Image file not found for pair {idx}. Returning None.")
            return None

        if self.transform:
            img1 = self.transform(img1) if img1 else None
            img2 = self.transform(img2) if img2 else None

        return img1, img2, torch.tensor(label, dtype=torch.float32)

# Define the transform
transform = transforms.Compose([
    transforms.Resize((112, 112)),
    transforms.ToTensor()
])

# Initialize the dataset and dataloader
image_folder = 'lfw_cropped/lfw_cropped'  # Update with the path to your unzipped dataset folder
pairs_file = 'pairs.txt'  # Update with the path to pairs.txt
test_dataset = RealTestDataset(image_folder, pairs_file, transform=transform)

# Filter out pairs with missing images and create DataLoader
filtered_pairs = [pair for pair in test_dataset if pair is not None]
filtered_dataset = [x for x in filtered_pairs if x is not None]
test_loader = DataLoader(filtered_dataset, batch_size=1, shuffle=False)

# Load the saved model weights
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SiameseNetwork().to(device)
model.load_state_dict(torch.load('golden_networks/networks-siamese/final_network.pth'))
model.eval()

# Evaluation function
def evaluate(model, data_loader):
    model.eval()
    mated_scores = []
    non_mated_scores = []
    with torch.no_grad():
        for img1, img2, label in data_loader:
            if img1 is None or img2 is None:
                # Handle missing images: Skip or assign a placeholder score
                continue
            
            img1, img2, label = img1.to(device), img2.to(device), label.to(device)
            outputs = model(img1, img2).squeeze()
            score = torch.sigmoid(outputs).item()
            if label.item() == 1:
                mated_scores.append(score)
            else:
                non_mated_scores.append(score)
    
    # Pad the shorter list with -1 for uneven lengths (optional)
    max_len = max(len(mated_scores), len(non_mated_scores))
    mated_scores.extend([-1] * (max_len - len(mated_scores)))
    non_mated_scores.extend([-1] * (max_len - len(non_mated_scores)))
    
    return mated_scores, non_mated_scores

# Evaluate on the real test set
mated_scores, non_mated_scores = evaluate(model, test_loader)

# Save mated and non-mated scores to .txt files
mated_scores_file = 'siamesescores/mated_scores.txt'
non_mated_scores_file = 'siamesescores/non_mated_scores.txt'

with open(mated_scores_file, 'w') as f:
    for score in mated_scores:
        f.write(f'{score}\n')

with open(non_mated_scores_file, 'w') as f:
    for score in non_mated_scores:
        f.write(f'{score}\n')

print(f'Mated Scores: {mated_scores[:5]}')
print(f'Non-Mated Scores: {non_mated_scores[:5]}')

with zipfile.ZipFile('siamesescores/predictions.zip', 'w') as zipf:
    zipf.write(mated_scores_file)
    zipf.write(non_mated_scores_file)
    
# Calculate accuracy for mated scores (> 0.5)
mated_correct = sum(score > 0.5 for score in mated_scores)
mated_accuracy = mated_correct / len(mated_scores) * 100

# Calculate accuracy for non-mated scores (< 0.5)
non_mated_correct = sum(score < 0.5 for score in non_mated_scores)
non_mated_accuracy = non_mated_correct / len(non_mated_scores) * 100

print(f"Mated Accuracy: {mated_accuracy:.2f}%")
print(f"Non-Mated Accuracy: {non_mated_accuracy:.2f}%")


C:\Users\phili\AppData\Local\Temp\ipykernel_7880\2228442471.py:56: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('golden_networks/networks-s

Mated Scores: [0.9458690285682678, 0.9680026173591614, 0.05446825921535492, 0.5319024324417114, 0.9370802044868469]
Non-Mated Scores: [0.07858853042125702, 6.347186717903242e-05, 0.24736875295639038, 8.3736922533717e-05, 0.9689295291900635]
Mated Accuracy: 56.37%
Non-Mated Accuracy: 85.00%


# Testing of Triplet

In [31]:
class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        # Convolutional layers
        self.conv1 = nn.Conv2d(1, 8, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(8)
        self.conv2 = nn.Conv2d(8, 16, kernel_size=5, padding=0)
        self.bn2 = nn.BatchNorm2d(16)
        self.conv3 = nn.Conv2d(16, 32, kernel_size=3, padding=0)
        self.bn3 = nn.BatchNorm2d(32)

        # Fully connected layers
        self.fc1 = nn.Linear(32 * 12 * 12, 41)  # Updated to 32 * 12 * 12
        self.fc1komma5 = nn.Linear(41,32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 1)
        self.dropout = nn.Dropout(0.5)

    def forward_one(self, x):
        x = F.relu(self.conv1(x)) # 8 * 112 * 112
        x = F.max_pool2d(x, 2)  # output size: (8, 56, 56)
        x = F.relu(self.conv2(x)) # 16* 52 * 52
        x = F.max_pool2d(x, 2)  # output size: (16, 26, 26)
        x = F.relu(self.conv3(x)) # 32 * 24 * 24
        x = F.max_pool2d(x, 2)  # output size: (32, 12, 12)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc1komma5(x))
        x = F.relu(self.fc2(x))
        return x

    def forward(self, input1, input2, input3):
        output1 = self.forward_one(input1)
        output2 = self.forward_one(input2)
        output3 = self.forward_one(input3)
        return output1, output2, output3

In [32]:
# Load the trained model
device = torch.device("cpu")
model = SiameseNetwork().to(device)
model.load_state_dict(torch.load('triplet/network_epoch1.pth'))
model.eval()
 
# Define the transformation (should match the transformation used during training)
transform = transforms.Compose([
    transforms.Resize((112, 112)),
    transforms.ToTensor(),
])
 
def preprocess_image(image_path):
    image = Image.open(image_path).convert('L')
    image = transform(image)
    image = image.unsqueeze(0)  # Add batch dimension
    return image
 
def compute_similarity(model, image1, image2, alpha=0.155):
    with torch.no_grad():
        embedding1 = model.forward_one(image1)
        embedding2 = model.forward_one(image2)
        distance = torch.mean(F.pairwise_distance(embedding1, embedding2))
        similarity = torch.exp(-alpha * distance)
        #similarity = 1 / (1 + torch.exp(alpha * distance))
    return similarity.item()
    #return distance
 

class RealTestDataset(Dataset):
    def __init__(self, image_folder, pairs_file, transform=None):
        self.image_folder = image_folder
        self.pairs = self._read_pairs(pairs_file)
        self.transform = transform

    def _read_pairs(self, pairs_file):
        pairs = []
        with open(pairs_file, 'r') as file:
            for line in file:
                name_1, number_1, name_2, number_2 = line.strip().split()
                img1_path = os.path.join(self.image_folder, name_1, f'{name_1}_{int(number_1):04d}.png')
                img2_path = os.path.join(self.image_folder, name_2, f'{name_2}_{int(number_2):04d}.png')
                label = 1 if name_1 == name_2 else 0
                pairs.append((img1_path.replace('\\', '/'), img2_path.replace('\\', '/'), label))
        return pairs

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        img1_path, img2_path, label = self.pairs[idx]
        try:
            img1 = Image.open(img1_path).convert('L')
            img2 = Image.open(img2_path).convert('L')
        except FileNotFoundError:
            # Handle missing image files
            print(f"Warning: Image file not found for pair {idx}. Returning placeholder tensors.")
            img1 = Image.new('L', (112, 112))  # Create a black image placeholder
            img2 = Image.new('L', (112, 112))  # Create a black image placeholder

        if self.transform:
            img1 = self.transform(img1)
            img2 = self.transform(img2)

        return img1, img2, torch.tensor(label, dtype=torch.float32)


# Define the transform
transform = transforms.Compose([
    transforms.Resize((112, 112)),
    transforms.ToTensor()
])

# Initialize the dataset and dataloader
image_folder = 'lfw_cropped/lfw_cropped'  # Update with the path to your unzipped dataset folder
pairs_file = 'pairs.txt'  # Update with the path to pairs.txt
test_dataset = RealTestDataset(image_folder, pairs_file, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

# Evaluation function
def evaluate(model, data_loader):
    model.eval()
    mated_scores = []
    non_mated_scores = []
    all_scores = []
    all_labels = []
    total_pairs = len(data_loader.dataset)

    print(f"Evaluating {total_pairs} pairs...")
    for idx, (img1, img2, label) in enumerate(data_loader):
        if img1 is None or img2 is None:
            mated_scores.append(-1)
            non_mated_scores.append(-1)
            continue
        
        img1, img2, label = img1.to(device), img2.to(device), label.to(device)
        # Compute similarity
        score = compute_similarity(model, img1, img2)
        
        all_scores.append(score)
        all_labels.append(label.item())

        if label.item() == 1:
            mated_scores.append(score)
        else:
            non_mated_scores.append(score)

        # Print progress
        if (idx + 1) % 100 == 0 or (idx + 1) == total_pairs:
            print(f"Progress: {idx + 1}/{total_pairs}")

    return mated_scores, non_mated_scores, all_scores, all_labels

# Evaluate on the real test set specified in pairs.txt
mated_scores, non_mated_scores, all_scores, all_labels = evaluate(model, test_loader)

# Save mated and non-mated scores to .txt files
mated_scores_file = 'tripletscores/mated_scores.txt'
non_mated_scores_file = 'tripletscores/non_mated_scores.txt'

with open(mated_scores_file, 'w') as f:
    for score in mated_scores:
        f.write(f'{score}\n')

with open(non_mated_scores_file, 'w') as f:
    for score in non_mated_scores:
        f.write(f'{score}\n')

print(f'Mated Scores: {mated_scores[:5]}')
print(f'Non-Mated Scores: {non_mated_scores[:5]}')

# Zip the files
zip_filename = 'tripletscores/predictions.zip'
with zipfile.ZipFile(zip_filename, 'w') as zipf:
    zipf.write(mated_scores_file)
    zipf.write(non_mated_scores_file)

print(f"Predictions saved to {zip_filename}")

# Calculate accuracy for mated scores (> 0.5)
mated_correct = sum(score > 0.5 for score in mated_scores)
mated_accuracy = mated_correct / len(mated_scores) * 100

# Calculate accuracy for non-mated scores (< 0.5)
non_mated_correct = sum(score < 0.5 for score in non_mated_scores)
non_mated_accuracy = non_mated_correct / len(non_mated_scores) * 100

print(f"Mated Accuracy: {mated_accuracy:.2f}%")
print(f"Non-Mated Accuracy: {non_mated_accuracy:.2f}%")

# Calculate mean absolute error (MAE)
mae = sum(abs(score - label) for score, label in zip(all_scores, all_labels)) / len(all_labels)
print(f"Mean Absolute Error (MAE): {mae:.4f}")

C:\Users\phili\AppData\Local\Temp\ipykernel_7880\2571125585.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('triplet/network_epoch1.pth'

Evaluating 6000 pairs...
Progress: 100/6000
Progress: 200/6000
Progress: 300/6000
Progress: 400/6000
Progress: 500/6000
Progress: 600/6000
Progress: 700/6000
Progress: 800/6000
Progress: 900/6000
Progress: 1000/6000
Progress: 1100/6000
Progress: 1200/6000
Progress: 1300/6000
Progress: 1400/6000
Progress: 1500/6000
Progress: 1600/6000
Progress: 1700/6000
Progress: 1800/6000
Progress: 1900/6000
Progress: 2000/6000
Progress: 2100/6000
Progress: 2200/6000
Progress: 2300/6000
Progress: 2400/6000
Progress: 2500/6000
Progress: 2600/6000
Progress: 2700/6000
Progress: 2800/6000
Progress: 2900/6000
Progress: 3000/6000
Progress: 3100/6000
Progress: 3200/6000
Progress: 3300/6000
Progress: 3400/6000
Progress: 3500/6000
Progress: 3600/6000
Progress: 3700/6000
Progress: 3800/6000
Progress: 3900/6000
Progress: 4000/6000
Progress: 4100/6000
Progress: 4200/6000
Progress: 4300/6000
Progress: 4400/6000
Progress: 4500/6000
Progress: 4600/6000
Progress: 4700/6000
Progress: 4800/6000
Progress: 4900/6000
Prog

# Testing Quadruplet

In [27]:
class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        # Convolutional layers
        self.conv1 = nn.Conv2d(1, 8, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(8)
        self.conv2 = nn.Conv2d(8, 16, kernel_size=5, padding=0)
        self.bn2 = nn.BatchNorm2d(16)
        self.conv3 = nn.Conv2d(16, 32, kernel_size=3, padding=0)
        self.bn3 = nn.BatchNorm2d(32)

        # Fully connected layers
        self.fc1 = nn.Linear(32 * 12 * 12, 41)  # Updated to 32 * 12 * 12
        self.fc1komma5 = nn.Linear(41,32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 1)
        self.dropout = nn.Dropout(0.5)

    def forward_one(self, x):
        x = F.relu(self.conv1(x)) # 8 * 112 * 112
        x = F.max_pool2d(x, 2)  # output size: (8, 56, 56)
        x = F.relu(self.conv2(x)) # 16* 52 * 52
        x = F.max_pool2d(x, 2)  # output size: (16, 26, 26)
        x = F.relu(self.conv3(x)) # 32 * 24 * 24
        x = F.max_pool2d(x, 2)  # output size: (32, 12, 12)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc1komma5(x))
        x = F.relu(self.fc2(x))
        return x

    def forward(self, input1, input2, input3, input4):
        output1 = self.forward_one(input1)
        output2 = self.forward_one(input2)
        output3 = self.forward_one(input3)
        output4 = self.forward_one(input4)
        return output1, output2, output3, output4

In [30]:
# Load the trained model
device = torch.device("cpu")
model = SiameseNetwork().to(device)
model.load_state_dict(torch.load('networks_christoph_quadro/network_epoch10.pth'))
model.eval()
 
# Define the transformation (should match the transformation used during training)
transform = transforms.Compose([
    transforms.Resize((112, 112)),
    transforms.ToTensor(),
])
 
def preprocess_image(image_path):
    image = Image.open(image_path).convert('L')
    image = transform(image)
    image = image.unsqueeze(0)  # Add batch dimension
    return image
 
def compute_similarity(model, image1, image2, alpha=0.155):
    with torch.no_grad():
        embedding1 = model.forward_one(image1)
        embedding2 = model.forward_one(image2)
        distance = torch.mean(F.pairwise_distance(embedding1, embedding2))
        similarity = torch.exp(-alpha * distance)
        #similarity = 1 / (1 + torch.exp(alpha * distance))
    return similarity.item()
    #return distance
 

class RealTestDataset(Dataset):
    def __init__(self, image_folder, pairs_file, transform=None):
        self.image_folder = image_folder
        self.pairs = self._read_pairs(pairs_file)
        self.transform = transform

    def _read_pairs(self, pairs_file):
        pairs = []
        with open(pairs_file, 'r') as file:
            for line in file:
                name_1, number_1, name_2, number_2 = line.strip().split()
                img1_path = os.path.join(self.image_folder, name_1, f'{name_1}_{int(number_1):04d}.png')
                img2_path = os.path.join(self.image_folder, name_2, f'{name_2}_{int(number_2):04d}.png')
                label = 1 if name_1 == name_2 else 0
                pairs.append((img1_path.replace('\\', '/'), img2_path.replace('\\', '/'), label))
        return pairs

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        img1_path, img2_path, label = self.pairs[idx]
        try:
            img1 = Image.open(img1_path).convert('L')
            img2 = Image.open(img2_path).convert('L')
        except FileNotFoundError:
            # Handle missing image files
            print(f"Warning: Image file not found for pair {idx}. Returning placeholder tensors.")
            img1 = Image.new('L', (112, 112))  # Create a black image placeholder
            img2 = Image.new('L', (112, 112))  # Create a black image placeholder

        if self.transform:
            img1 = self.transform(img1)
            img2 = self.transform(img2)

        return img1, img2, torch.tensor(label, dtype=torch.float32)


# Define the transform
transform = transforms.Compose([
    transforms.Resize((112, 112)),
    transforms.ToTensor()
])

# Initialize the dataset and dataloader
image_folder = 'lfw_cropped/lfw_cropped'  # Update with the path to your unzipped dataset folder
pairs_file = 'pairs.txt'  # Update with the path to pairs.txt
test_dataset = RealTestDataset(image_folder, pairs_file, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

# Evaluation function
def evaluate(model, data_loader):
    model.eval()
    mated_scores = []
    non_mated_scores = []
    all_scores = []
    all_labels = []
    total_pairs = len(data_loader.dataset)

    print(f"Evaluating {total_pairs} pairs...")
    for idx, (img1, img2, label) in enumerate(data_loader):
        if img1 is None or img2 is None:
            mated_scores.append(-1)
            non_mated_scores.append(-1)
            continue
        
        img1, img2, label = img1.to(device), img2.to(device), label.to(device)
        # Compute similarity
        score = compute_similarity(model, img1, img2)
        
        all_scores.append(score)
        all_labels.append(label.item())

        if label.item() == 1:
            mated_scores.append(score)
        else:
            non_mated_scores.append(score)

        # Print progress
        if (idx + 1) % 100 == 0 or (idx + 1) == total_pairs:
            print(f"Progress: {idx + 1}/{total_pairs}")

    return mated_scores, non_mated_scores, all_scores, all_labels

# Evaluate on the real test set specified in pairs.txt
mated_scores, non_mated_scores, all_scores, all_labels = evaluate(model, test_loader)

# Save mated and non-mated scores to .txt files
mated_scores_file = 'quadruplescores/mated_scores.txt'
non_mated_scores_file = 'quadruplescores/non_mated_scores.txt'

with open(mated_scores_file, 'w') as f:
    for score in mated_scores:
        f.write(f'{score}\n')

with open(non_mated_scores_file, 'w') as f:
    for score in non_mated_scores:
        f.write(f'{score}\n')

print(f'Mated Scores: {mated_scores[:5]}')
print(f'Non-Mated Scores: {non_mated_scores[:5]}')

# Zip the files
zip_filename = 'quadruplescores/predictions.zip'
with zipfile.ZipFile(zip_filename, 'w') as zipf:
    zipf.write(mated_scores_file)
    zipf.write(non_mated_scores_file)

print(f"Predictions saved to {zip_filename}")

# Calculate accuracy for mated scores (> 0.5)
mated_correct = sum(score > 0.5 for score in mated_scores)
mated_accuracy = mated_correct / len(mated_scores) * 100

# Calculate accuracy for non-mated scores (< 0.5)
non_mated_correct = sum(score < 0.5 for score in non_mated_scores)
non_mated_accuracy = non_mated_correct / len(non_mated_scores) * 100

print(f"Mated Accuracy: {mated_accuracy:.2f}%")
print(f"Non-Mated Accuracy: {non_mated_accuracy:.2f}%")

# Calculate mean absolute error (MAE)
mae = sum(abs(score - label) for score, label in zip(all_scores, all_labels)) / len(all_labels)
print(f"Mean Absolute Error (MAE): {mae:.4f}")

C:\Users\phili\AppData\Local\Temp\ipykernel_7880\506473446.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('networks_christoph_quadro/ne

Evaluating 6000 pairs...
Progress: 100/6000
Progress: 200/6000
Progress: 300/6000
Progress: 400/6000
Progress: 500/6000
Progress: 600/6000
Progress: 700/6000
Progress: 800/6000
Progress: 900/6000
Progress: 1000/6000
Progress: 1100/6000
Progress: 1200/6000
Progress: 1300/6000
Progress: 1400/6000
Progress: 1500/6000
Progress: 1600/6000
Progress: 1700/6000
Progress: 1800/6000
Progress: 1900/6000
Progress: 2000/6000
Progress: 2100/6000
Progress: 2200/6000
Progress: 2300/6000
Progress: 2400/6000
Progress: 2500/6000
Progress: 2600/6000
Progress: 2700/6000
Progress: 2800/6000
Progress: 2900/6000
Progress: 3000/6000
Progress: 3100/6000
Progress: 3200/6000
Progress: 3300/6000
Progress: 3400/6000
Progress: 3500/6000
Progress: 3600/6000
Progress: 3700/6000
Progress: 3800/6000
Progress: 3900/6000
Progress: 4000/6000
Progress: 4100/6000
Progress: 4200/6000
Progress: 4300/6000
Progress: 4400/6000
Progress: 4500/6000
Progress: 4600/6000
Progress: 4700/6000
Progress: 4800/6000
Progress: 4900/6000
Prog